### LVV-T1846: Verify calculation of band-to-band color zero-point accuracy including u-band

Verify that the DM system provides software to assess whether the accuracy of absolute band-to-band color zero-points for all colors constructed from any filter pair, including the u-band, is less than **PA5u = 10 millimagnitudes**.

**Discussion**: This requires observations of a source of "truth" in the form of [CalSpec](https://www.stsci.edu/hst/instrumentation/reference-data-for-calibration-and-tools/astronomical-catalogs/calspec) (or similar) spectrophotometric standards that can be used to assess the accuracy. At the time of this testing, only a single CalSpec standard (C26202) has been observed in all LSST bands, and only with LSSTComCam. This notebook demonstrates the methods for C26202 as observed with LSSTComCam and included in Data Preview 1 (DP1); once sufficient LSSTCam observations become available, this analysis can be done for LSSTCam data.

In [1]:
# Generic python packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import math
import os
import gc
import warnings

# Set the environment variable to point to the rubin_sim data:
os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

# LSST Science Pipelines (Stack) packages
import lsst.daf.butler as dafButler

# rubin_sim-related packages
import rubin_sim.phot_utils as pt
import syseng_throughputs as st
from rubin_sim.data import get_data_dir

# Astropy-related packages
from astropy import units as u
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.table import Table

# Set filter warnings to "ignore" to avoid a lot of excess log messages to the screen:
warnings.filterwarnings("ignore")

/sdf/data/rubin/user/jcarlin/repos/syseng_throughputs/syseng_throughputs/bandpassUtils.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
# Which repo, collection, instrument, and skymap to use.
repo = '/repo/dp1'
collections = 'LSSTComCam/DP1'

instrument = 'LSSTComCam'
skymap_name = 'lsst_cells_v1'
day_obs_start = 20241101
day_obs_end = 20241231

verbose = 1   # 0, 1, 2, ...  Larger means more output to the screen.

# Which flux to use?  psfFlux or calibFlux?
fluxName = 'psfFlux'
fluxerrName = 'psfFluxErr'
#fluxName = 'calibFlux'
#fluxerrName = 'calibFluxErr'

# CalSpec star name
star_name = 'C26202'

# Which CalSpec spectrum FITS files to to use?
sedfile_dict = {'stiswfcnic_007' : '/sdf/data/rubin/shared/calspec/2024-11-08/c26202_stiswfcnic_007.fits', 
                'mod_008'        : '/sdf/data/rubin/shared/calspec/2024-11-08/c26202_mod_008.fits'
               }

# RA, DEC of calspec star in degrees
raDeg = 53.136845833333325
decDeg = -27.86349444444444

# List of filters to examine
flist = ['u', 'g', 'r', 'i', 'z', 'y']

#### Calculate Synthetic AB magnitudes for CalSpec star, based on official filter bandpasses

Set up appropriate hardware and system from `syseng_throughputs`

In [3]:
defaultDirs = st.setDefaultDirs()
if instrument == "LSSTComCam":
    #Change detectors from (default) LSST to ComCam (ITL CCDs)
    defaultDirs['detector'] = defaultDirs['detector'].replace('/joint_minimum',
                                                              '/itl')
hardware, system = st.buildHardwareAndSystem(defaultDirs)

#### Calculate synthetic mags for C26202

These are calculated for two different calibrated spectra of the CalSpec star: 'stiswfcnic_007' is the observed HST SED, and 'mod_008' is a model SED fit to the HST observations.

In [4]:
mags = {}

# Loop through the SEDs in our sedfile dictionary
for sed_key in sedfile_dict:
    
    print(sed_key, sedfile_dict[sed_key])
    
    # Read the SED file associated with this SED
    sedfile = sedfile_dict[sed_key]
    seddata = fits.getdata(sedfile)

    # Transform the SED data into rubin_sim format
    wavelen = seddata['WAVELENGTH'] * u.angstrom.to(u.nanometer) # This is in angstroms - need in nanometers
    flambda = seddata['FLUX'] / (u.angstrom.to(u.nanometer)) # this is in erg/sec/cm^^2/ang but we want /nm     
    sed = pt.Sed(wavelen=wavelen, flambda=flambda)
    
    # Loop over the filters, calculating the synthetic mags for each filter for this SED
    mags[sed_key] = []
    for f in flist:
        # Append the synthetic mag for this filter to this mags list for this SED
        mags[sed_key].append(sed.calc_mag(system[f]))
    # Convert list of synthetic mags for this SED into a numpy array
    mags[sed_key] = np.array(mags[sed_key])

stiswfcnic_007 /sdf/data/rubin/shared/calspec/2024-11-08/c26202_stiswfcnic_007.fits
mod_008 /sdf/data/rubin/shared/calspec/2024-11-08/c26202_mod_008.fits


In [5]:
# Convert the mags numpy arrays into a pandas dataframe
df_mags = pd.DataFrame(mags, index=flist)
df_mags

,stiswfcnic_007,mod_008
u,17.572800,17.586964
g,16.691931,16.692687
r,16.362017,16.361654
i,16.260196,16.259542
z,16.243679,16.243690
y,16.238847,16.238887


#### Query the Butler for observations of the CalSpec star

In [6]:
butler = dafButler.Butler(repo, collections=collections)

Find all `visit_image`s that overlap the sky position of the CalSpec star.

In [7]:
datasetRefs = butler.query_datasets("object", where="patch.region OVERLAPS POINT(ra, dec)",
                                    bind={"ra": raDeg, "dec": decDeg})

for i, ref in enumerate(datasetRefs):    
    print(i, ref.dataId)
    if ((verbose < 2) & (i >= 10)): 
        print("...")
        break

print(f"\nFound {len(datasetRefs)} object tables")

0 {skymap: 'lsst_cells_v1', tract: 5063}

Found 1 object tables


#### Create a pandas DataFrame containing the `object` info for the standard(s).

Loop over the `datasetRefs`, grab the contents of the `object` table for each `ref`, select the entries within 3 arcsec of the Calspec star, and combine them all into one big pandas DataFrame.  

In [8]:
obj_list = []

# Create SkyCoord object for the coordinates of CalSpec star
ref_coord = SkyCoord(ra=raDeg*u.degree, dec=decDeg*u.degree)

for i, ref in enumerate(datasetRefs):
    
    # Retrieve the Object table for this ref...
    # dataId = {'tract': ref.dataId['tract'], 'detector': ref.dataId['detector']}
    obj = butler.get(ref, storageClass='DataFrame')

    # Create SkyCoord object for all points in the dataframe
    obj_coords = SkyCoord(ra=obj['coord_ra'].values*u.degree, 
                          dec=obj['coord_dec'].values*u.degree)

    # Calculate separations
    separations = ref_coord.separation(obj_coords)

    # Create mask for points within 3.0 arcseconds
    mask_sep = separations < 3.0*u.arcsec

    # Get filtered dataframe
    nearby_good_df = obj[mask_sep]

    # Include the separations in the result
    orig_columns = nearby_good_df.columns
    nearby_good_df = obj[mask_sep].copy()
    nearby_good_df['separation_calspec'] = separations[mask_sep].arcsec

    # Find (and keep) the closet match within the match radius
    best_df = nearby_good_df.sort_values('separation_calspec').drop_duplicates(subset=orig_columns,
                                                                               keep='first')
    obj_list.append(best_df)

    if ((verbose >= 2) | (i < 10)): 
        print(f"{i} Visit {ref.dataId}:  Retrieved catalog of {len(obj)} sources.")
    if ((verbose < 2) & (i == 10)): 
        print("...")

obj_all = pd.concat(obj_list, ignore_index=True)

print("")
print(f"Total combined catalog contains {len(obj_all)} objects.")


0 Visit {skymap: 'lsst_cells_v1', tract: 5063}:  Retrieved catalog of 335955 sources.

Total combined catalog contains 1 objects.


Add mag_obs and mag_obsErr columns:

In [9]:
for band in flist:
    # Flux in nano-Janskys to AB magnitudes:
    obj_all[f"{band}mag_obs"] = -2.5*np.log10(obj_all[f"{band}_{fluxName}"]) + 31.4

    # Flux error in nano-Janskys to AB magnitude error:
    # Factor of 2.5/math.log(10) is explained here:  https://astronomy.stackexchange.com/questions/38371/how-can-i-calculate-the-uncertainties-in-magnitude-like-the-cds-does
    obj_all[f"{band}mag_obsErr"] = 2.5/math.log(10)*obj_all[f"{band}_{fluxerrName}"]/obj_all[f"{band}_{fluxName}"]

Look at the resulting table:

In [10]:
obj_all

,tract,patch,g_ra,g_dec,g_raErr,g_decErr,g_ra_dec_Cov,g_psfFlux,g_psfFluxErr,g_free_psfFlux,...,gmag_obs,gmag_obsErr,rmag_obs,rmag_obsErr,imag_obs,imag_obsErr,zmag_obs,zmag_obsErr,ymag_obs,ymag_obsErr
0,5063,24,53.137028,-27.863438,1.610198e-08,1.717744e-08,1.427354e-20,761098.625,63.617188,761088.25,...,16.696396,0.000091,16.363503,0.000076,16.260647,0.000094,16.244568,0.000117,16.239956,0.00045


#### Calculate the colors and compare to the standard(s)

Calculate various colors, then compare them to the spectrophotometric standard's colors.

In [11]:
colors_list = [('u', 'g'), ('u', 'r'), ('u', 'i'), ('u', 'z'), ('u', 'y')]

In [12]:
colornames = []
meas_color = []
hst_color = []
model_color = []

for bands in colors_list:
    mag1 = obj_all[f"{bands[0]}mag_obs"]
    mag2 = obj_all[f"{bands[1]}mag_obs"]
    std_row1 = df_mags[df_mags.index == bands[0]]
    std_row2 = df_mags[df_mags.index == bands[1]]

    colornames.append(f"{bands[0]}-{bands[1]}")
    meas_color.append(mag1 - mag2)
    hst_color.append(std_row1['stiswfcnic_007'].values - std_row2['stiswfcnic_007'].values)
    model_color.append(std_row1['mod_008'].values - std_row2['mod_008'].values)


colornames = np.array(colornames)
meas_color = np.array(meas_color)
hst_color = np.array(hst_color)
model_color = np.array(model_color)
hst_color_diff = 1000.0*np.array(meas_color - hst_color)
model_color_diff = 1000.0*np.array(meas_color - model_color)

tab = Table([colornames, meas_color, hst_color, model_color, hst_color_diff, model_color_diff],
            names = ['color', 'meas_DP1', 'stiswfcnic_007', 'mod_008', 'diff_DP1_stiswfcnic_007', 'diff_DP1_mod_008'],
            units = ['', u.mag, u.mag, u.mag, u.mmag, u.mmag])

for col in tab.columns[1:]:
    tab[col].format = '%.4F'

In [13]:
tab

color,meas_DP1,stiswfcnic_007,mod_008,diff_DP1_stiswfcnic_007,diff_DP1_mod_008
,mag,mag,mag,mmag,mmag
str3,float32[1],float64[1],float64[1],float64[1],float64[1]
u-g,0.9277,0.8809,0.8943,46.8064,33.3986
u-r,1.2606,1.2108,1.2253,49.7860,35.2593
u-i,1.3634,1.3126,1.3274,50.8198,36.0025
u-z,1.3795,1.3291,1.3433,50.3817,36.2290
u-y,1.3841,1.3340,1.3481,50.1626,36.0382


### Results:

We have demonstrated that tools exist to calculate the accuracy of band-to-band color zeropoints by comparison between measured magnitudes and expected magnitudes of a spectrophotometric flux standard. For LSSTComCam DP1 data, the offsets between colors from the calibrated `object` tables and the predicted colors are more than **PA5u = 10 mmag** for all colors that include the `u` band. Nonetheless, we deem the result of this test a **Pass**, as we have demonstrated that these offsets can be calculated as required. We will assess further with LSSTCam on-sky data when it becomes available.